In [1]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [22]:
#BAN HANG
URL_ban="../VACP/Export BRAVO/Sổ chi tiết bán hàng.xls"  
data_banhang=pd.read_excel(URL_ban,skipfooter=1,header=6).rename(
    columns={
        "Unnamed: 0":"Ngày bán",
        "Unnamed: 1":"ID Order",
        "Unnamed: 2":"ID Khách hàng",
        "Unnamed: 3":"Hãng SX",
        "Unnamed: 5":"ID",
        "Unnamed: 7":"Tên hàng",
        "Tiền bán":"Thành tiền",
        "Unnamed: 19":"Nhân viên",
        "Unnamed: 22":"ID Nhóm hàng"

    }
)

In [28]:
data_banhang=data_banhang[["Ngày bán","ID Order","ID Khách hàng","ID Nhóm hàng","Nhân viên","ID","Tên hàng","Số lượng","Giá bán","Thành tiền"]]

In [29]:
data_banhang

,Ngày bán,ID Order,ID Khách hàng,ID Nhóm hàng,Nhân viên,ID,Tên hàng,Số lượng,Giá bán,Thành tiền
0,2022-11-21,VAPC211122-001,PHUTUNGATSG,KIMPHUN,HOANVU,23209-09045,Kim phun 1TR,2,1400000.0,2800000
1,2022-11-21,VAPC211122-002,KHACHLE-HOANVU,CANBANG,HOANVU,SL-3880,Cân bằng trước Innova,2,190000.0,380000
2,2022-11-21,VAPC211122-002,KHACHLE-HOANVU,LAI,HOANVU,SR-3880,Lái trong 2TR,2,230000.0,460000
3,2022-11-21,VAPC211122-002,KHACHLE-HOANVU,LAI,HOANVU,SE-3881,Lái ngoài Innova,2,190000.0,380000
4,2022-11-21,VAPC211122-003,HAHAU,BIT3,VINHNGUYEN,3DACF027F-13S-AM,Bi moay ơ sau (hộp giấy xi măng),3,1380000.0,4140000
...,...,...,...,...,...,...,...,...,...,...
38769,2022-01-03,VAPC030122-096,THUANPHAT,GIAMSOC,HOANVU,48530-59177,Giảm sóc sau Vios,2,580000.0,1160000
38770,2022-01-03,VAPC030122-097,KL-TANTHEAUTO,THUOCLAI,HOANTRAN,45510-48040,Thước lái RX350 trợ lực điện,1,11300000.0,11300000
38771,2022-01-03,VAPC030122-098,CHPT-ANNHIEN,BOMNUOC,HOANTRAN,16100-39436,Bơm nước Lexus ( có 2 gioăng chỉ kèm ) (liền g...,1,2450000.0,2450000
38772,2022-01-03,VAPC030122-099,KL-NHATBENTHANH,DAY,HOANVU,90916-T2026,Dây tăng tổng 7PK1935,1,380000.0,380000


In [27]:
data_banhang.groupby(["Ngày bán","ID Khách hàng"]).sum("Thanh tiền")

Unnamed: 9  Số lượng  Đơn giá gốc    Giá bán  \
Ngày bán   ID Khách hàng                                                  
2022-01-03 AEA                   3933         3    1520000.0  1540000.0   
           ANHAN                 1311         2     240000.0   250000.0   
           ANHCHINH-AT           5244         8    1683000.0  1665000.0   
           ANHCUONG              1311         1    2200000.0  2600000.0   
           ANHTAN                7866         6    5430000.0  5420000.0   
...                               ...       ...          ...        ...   
2022-11-21 KL-HIEUPHAT-TM        2622        10    1100000.0  1060000.0   
           KL-KIMCUONG           1311         1     340000.0   340000.0   
           KL-MINHPHAT           2622         2    2700000.0  2700000.0   
           KL-QUANGSAIGON        1311         1     142000.0   140000.0   
           PHUTUNGATSG           1311         2    1400000.0  1400000.0   

                           Chênh lệch  Thành tiền  Unnamed: 15  Unnamed: 18  \
Ngày bán   ID Khách hàng                                                      
2022-01-03 AEA                  20000     1540000       0.1667          0.0   
           ANHAN                10000      500000       0.0417          0.0   
           ANHCHINH-AT           2000     2945000       0.0025          0.0   
           ANHCUONG            400000     2600000       0.1818          0.0   
           ANHTAN              -10000     5420000      -0.0116          0.0   
...                               ...         ...          ...          ...   
2022-11-21 KL-HIEUPHAT-TM           0     5300000       0.0000          0.0   
           KL-KIMCUONG              0      340000       0.0000          0.0   
           KL-MINHPHAT              0     2700000       0.0000          0.0   
           KL-QUANGSAIGON           0      140000       0.0000          0.0   
           PHUTUNGATSG              0     2800000       0.0000          0.0   

                           Unnamed: 20  Unnamed: 21  
Ngày bán   ID Khách hàng                             
2022-01-03 AEA                     0.0          0.0  
           ANHAN                   0.0          0.0  
           ANHCHINH-AT             0.0          0.0  
           ANHCUONG                0.0          0.0  
           ANHTAN                  0.0          0.0  
...                                ...          ...  
2022-11-21 KL-HIEUPHAT-TM          0.0          0.0  
           KL-KIMCUONG             0.0          0.0  
           KL-MINHPHAT             0.0          0.0  
           KL-QUANGSAIGON          0.0          0.0  
           PHUTUNGATSG             0.0          0.0  

[12656 rows x 10 columns]